# Описание проекта

Интернет-магазин запускает новый сервис: теперь пользователи могут сами редактировать и дополнять описания товаров. Необходимо разработать инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

### Метрики

*F1* > 0,75. 

# 1. Подготовка

## Подготовка среды

Импортируем необходимые библиотеки.

In [1]:
%%time

import pandas as pd
import numpy as np
import itertools
import json

import torch
import transformers
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re
from tqdm import notebook
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import catboost as cb

from IPython.display import display as d
import warnings

Wall time: 16 s


Настроим среду.

In [2]:
warnings.filterwarnings('ignore')
r_state = 123

URL-адрес модели *BERT*.

conversational_cased_L-12_H-768_A-12_pt

http://files.deeppavlov.ai/deeppavlov_data/bert/conversational_cased_L-12_H-768_A-12_pt.tar.gz

## Функции

In [3]:
# The function removes unnecessary spaces

def space_reduction(corpus):
    for i in range(len(corpus)):
        corpus[i] = corpus[i].split()
        corpus[i] = ' '.join(corpus[i])

In [4]:
# The function counts the difference between the balances of classes of the original and the current sample

def class_ratio_control(df, test_size=1/6, precision=0.95):
    original = df.target[df.target == 1].count() / \
               df.target[df.target == 0].count()

    while True:
        index_train, index_test = train_test_split(df.index.values, test_size=test_size)
        
        train = df.loc[index_train, 'target'][df.target == 1].count() / \
                df.loc[index_train, 'target'][df.target == 0].count()

        test = df.loc[index_test, 'target'][df.target == 1].count() / \
               df.loc[index_test, 'target'][df.target == 0].count()
        
        if ( (precision < train/original < 1/precision) and (precision < test/original < 1/precision) ):
            print('ratio of the classes:\noriginal: {:.4f}\ntrain:    {:.4f}\ntest:     {:.4f}\n'
                  .format(original, train, test)
                 )
            return index_train, index_test

In [5]:
# The function computes indexes of the train and the validation sample for cross-validation

def cv_indexes(index_train_valid, cv=3, valid_size=0.2):
    valid_size_loc = valid_size
    if cv * valid_size > 1.0:
        valid_size_loc = 0.98 / cv

    index_train = []
    index_valid = []

    for i in range(cv):
        bounds = [
            int(len(index_train_valid) * valid_size_loc * i),
            int(len(index_train_valid) * valid_size_loc * (i+1))
        ]

        if bounds[1] > len(index_train_valid):
            bounds[1] = len(index_train_valid)

        index_train.append(np.concatenate((index_train_valid[:bounds[0]],
                                           index_train_valid[bounds[1]:]),
                                          axis=0
                                         )
                          )
        index_valid.append(index_train_valid[bounds[0]:bounds[1]])
    
    return index_train, index_valid

In [6]:
# The function replaces embeddings (or tokens) longer than max_pos by zeros

def position_embeddings_filter(x, max_pos):
    if len(x) > max_pos:
        return 0
    else:
        return x

In [7]:
# The func takes a sample from the data passed

def sample_func(df, n_samples, target_col_name, frac_one, return_index=False):
    data_1 = df[df[target_col_name] == 1].sample(n=int(n_samples * frac_one), random_state=r_state)
    data_0 = df[df[target_col_name] == 0].sample(n=int(n_samples - data_1.shape[0]), random_state=r_state)
    data = data_1.append(data_0).sample(frac=1, random_state=r_state)
    
    if return_index:
        return data.index
    else:
        return data.reset_index(drop=True)

## Ознакомление с данными

Загрузим данные из файла.

In [8]:
data_raw = pd.read_csv('toxic_comments.csv')

In [9]:
data_raw.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [10]:
data_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Файл открылся корректно, пропусков в данных нет.

Проверим целевой признак на наличие дисбаланса.

In [11]:
data_raw.toxic.value_counts()

0    143346
1     16225
Name: toxic, dtype: int64

Имеется дисбаланс целевого признака. Будем это учитывать.

Найдём долю положительного класса.

In [12]:
frac_class_1 = data_raw.toxic[data_raw.toxic == 1].count() / data_raw.toxic.count()
frac_class_1

0.10167887648758234

## Формирование корпуса текстов

Сформируем корпус текстов. Для этого приведём значения столбца *text* к типу данных *Unicode* и сохраним массив с ними в отдельной переменной.

In [13]:
%%time

corp_raw = data_raw.text.values.astype('U')

Wall time: 2.01 s


In [14]:
corp_raw[0]

"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

## Обработка корпуса текстов

### Вводная часть

При работе с текстами под созданием признаков понимается преобразование текстов в векторы. Мы будем это делать двумя алгоритмами:

* *TF-IDF*;
* *BERT*.

Эти два алгоритма сильно различаются между собой. В первом случае требуется предобработка текстов, а затем алгоритм преобразования необходимо обучить (причём делать это придётся не однократно, а множество раз в ходе кросс-валидации). Второй подход предполагает использование заранее обученной модели (признаки сформируем однократно уже в данном разделе) и отсутствие необходимости в лемматизации текста (потребуется лишь базовая предобработка).

В данном разделе сначала выполним базовую предобработку корпуса текстов, необходимую для обоих алгоритмов. Затем - подготовим корпус для использования в *TF-IDF*. После этого - сформируем признаки с помощью алгоритма *BERT*.

Корпус текстов для *TF-IDF* будет называться ***corp_lemm*** (так как будет выполнена в том числе лемматизация), а для *BERT* - ***corp*** (будет выполнена только очистка от небуквенных символов и лишних пробелов).

### Очистка, токенизация

Очистим корпус от небуквенных символов при помощи регулярных выражений, удалим стоп-слова (слова, не несущие смысловой нагрузки: предлоги, частицы и т.п.) и выполним токенизацию - разбиение текста на отдельные слова. Токенизация необходима для корректной работы алгоритма *WordNetLemmatizer* библиотеки *nltk*, при помощи которого будет выполняться лемматизация. Токенизация и удаление стоп-слов будут выполнены средствами библиотеки *nltk*.

Удаление стоп-слов перед выполнением токенизации (и, соответственно, лемматизации) обусловлено следующими причинами.
* Токенизация, как оказалось, искажает некоторые стоп-слова таким образом, что они не будут распознаны. Например: "weren't" превращается в "were" и "n't". Первое - удалится (т.к. есть в списке), а второе - останется.
* Сократится машинное время на разметку частей речи - самую ресурсоёмкую операцию - благодаря уменьшению количества слов в корпусе.

Загрузим словать стоп-слов.

In [15]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Андрей\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Создадим список стоп-слов с пробелами в начале и в конце, чтобы можно было отыскать в тексте корпуса стоп-слова как отдельные слова, а не как составные части других слов.

In [16]:
stop_words_with_spaces = [''] * len(stop_words)
stop_words_list = list(stop_words)

for i in range(len(stop_words_with_spaces)):
    stop_words_with_spaces[i] = f' {stop_words_list[i]} '

stop_words_with_spaces.sort(key=len, reverse=True)
stop_words_with_spaces[:5]

[' themselves ', ' yourselves ', ' ourselves ', " should've ", " shouldn't "]

Создадим две переменные, в которые поместим корпусы текстов до и после лемматизации (соответственно *corp* и *corp_lemm*).

Выполним очистку от небуквенных символов (для обоих массивов), удаление стоп-слов, токенизацию (только для *corp_lemm*).

In [17]:
%%time

corp = [''] * len(corp_raw)
corp_lemm = corp.copy()

for i in notebook.tqdm(range(len(corp_raw))):
    corp[i] = re.sub(r'[^a-zA-Z\' ]', ' ', corp_raw[i])
    corp_lemm[i] = corp[i].lower()

    for sw in stop_words_with_spaces:
        if (sw in corp_lemm[i]) == True:
            corp_lemm[i] = corp_lemm[i].replace(sw, ' ')
        if corp_lemm[i].startswith(sw[1:],) == True:
            corp_lemm[i] = corp_lemm[i][len(sw[1:]):]
        if corp_lemm[i].endswith(sw[:-1]) == True:
            corp_lemm[i] = corp_lemm[i][:-len(sw[:-1])]

    
    corp_lemm[i] = nltk.word_tokenize(corp_lemm[i])


Wall time: 1min 28s


Удалим исходный корпус текстов (он больше не потребуется), чтобы освободить память.

In [18]:
del corp_raw

### Лемматизация

Укажем, к какой части речи относится каждое слово корпуса, при помощи метода *pos_tag_sents*. Это необходимо для корректного выполнения лемматизации. По умолчанию алгоритм *WordNetLemmatizer* воспринимает все слова как существительные, а потом "не знает", что делать со словами, которые существительными по факту не являются. Поэтому требуется явное указание.

In [19]:
%%time

corp_tagged = nltk.pos_tag_sents(corp_lemm, tagset='universal', lang='eng')

Wall time: 5min 27s


Выполним лемматизацию.

Несколько странно, что в рамках одной библиотеки функция возвращает результат в формате, не подходящем для его подачи на вход другой функции, для которой этот результат главным образом предназначался. Такая ситуация - с тегами, обозначающими часть речи. Чтобы решить эту проблему, выполним небольшое преобразование (возьмём только первый символ тега и приведём его к нижнему регистру). В случае подачи несовместимого тега (да, такое здесь встречается) установим тег "существительное" ('n') в ветке исключения.

In [20]:
wnl=WordNetLemmatizer()

In [21]:
%%time

for i in notebook.tqdm(range(len(corp_tagged))):
    corp_lemm[i] = ''
    for x in range(len(corp_tagged[i])):
        try:
            lemma = wnl.lemmatize(corp_tagged[i][x][0], corp_tagged[i][x][1][0].lower())
        except:
            lemma = wnl.lemmatize(corp_tagged[i][x][0], 'n')
        corp_lemm[i] += f' {lemma}'


Wall time: 35.4 s


Удалим корпус текстов, дополненный тегами pos (он больше не потребуется), чтобы освободить память.

In [22]:
del corp_tagged

### Удаление лишних пробелов

Удалим лишние пробелы с помощью функции *space_reduction*.

In [23]:
%%time

space_reduction(corp)
space_reduction(corp_lemm)

Wall time: 1.38 s


Примеры очищенных текстов до и после лемматизации.

In [24]:
d(corp[11111])
d(corp_lemm[11111])

"There is no reason to treat AFF and SF the same The neutrality policy which explicitly states that it is not to be confused with all views deserve equal time no matter how many or how few people hold them certainly requires nothing of the sort look it up yourself I won t do your homework for you Moreover the merits of AFF and the merits of SF are two entirely separate and independent things No amount of slamming AFF even if your charges are accurate which I doubt as your behaviour here strongly suggests your perceptions on such things cannot be trusted can ever by itself make SF worth including SF must stand or fall on its own merits There are many serious strikes against including SF none of which you have even tried to address First of all it borders on being a blog which is already an almost decisive point against it there was talk of banning links to blogs entirely at one point though it isn t actually policy to do so as far as I know certainly it s a blog has stood uncontested as

"reason treat aff sf neutrality policy explicitly state confused view deserve equal time matter many people hold certainly require nothing sort look homework moreover merit aff merit sf two entirely separate independent thing amount slam aff even charge accurate doubt behaviour strongly suggest perception thing can not trust ever make sf worth include sf must stand fall merit many serious strike include sf none even try address first border blog already almost decisive point talk ban link blog entirely one point though actually policy far know certainly blog stand uncontested sufficient reason delete link many page include one fight equally importantly read part sf though agree characterization see others go far call hate site blunt largely chronicle one person 's persecution complex say elsewhere verge conspiracy theory kooky sense painfully bad prose screechy hysterical tone write simply way quality site would even could credit actual content three point heck two sufficient reason le

Обработка корпуса текстов завершена: на основе исходного созданы два очищенных корпуса: с лемматизированными и нелемматизированными текстами.

### Формирование датафрейма

Создадим датафрейм, чтобы избежать проблем с индексацией разрозненных массивов. Перед этим убедимся, что длины корпусов текстов и длина столбца с классом совпадают.

In [25]:
len(corp) == len(corp_lemm) == len(data_raw.toxic.values)

True

In [26]:
data = pd.DataFrame(data=None,
                    columns=['corp', 'corp_lemm', 'target']
                   )

data['corp'] = corp
data['corp_lemm'] = corp_lemm
data['target'] = data_raw.toxic.values

data.head()

,corp,corp_lemm,target
0,Explanation Why the edits made under my userna...,explanation edits make username hardcore metal...,0
1,D'aww He matches this background colour I'm se...,d'aww match background colour i 'm seemingly s...,0
2,Hey man I'm really not trying to edit war It's...,hey man i 'm really try edit war guy constantl...,0
3,More I can't make any real suggestions on impr...,ca n't make real suggestion improvement wonder...,0
4,You sir are my hero Any chance you remember wh...,sir hero chance remember page that 's,0


### TF-IDF

*TF-IDF* - это условное название алгоритма создания признаков. Алгоритм аналогичен *Мешку слов*. Название же - условное потому, что на самом деле *TF-IDF* - это метрика оценки важности каждого слова в корпусе текстов: важность слова зависит от частоты его употребления в тексте, а также от количества других текстов корпуса, где это слово встречается ещё.

Алгоритм предполагает работу с лемматизированными текстами.

Поскольку при обучении моделей необходимо применять кросс-валидацию, а векторизация (преобразование текстов в векторы), в свою очередь, тоже предполагает обучение, необходимо выполнять её (векторизацию) на каждом этапе кросс-валидации на соответствующей обучающей выборке. Векторизацию валидационной выборки следует выполнять уже обученным алгоритмом (на каждом этапе - обученным заново).

Векторизацию корпуса текстов будем выполнять при помощи функции *TfidfVectorizer* библиотеки *scikit-learn*.

Инициализируем счётчик.

In [27]:
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

### BERT

#### Выбор модели

Укажем путь к модели на диске.

In [28]:
bert_path = 'D:/py/bert/conversational_cased_L-12_H-768_A-12_pt/'
bert_model_file = 'pytorch_model.bin'

d(bert_path)
d(bert_model_file)

'D:/py/bert/conversational_cased_L-12_H-768_A-12_pt/'

'pytorch_model.bin'

#### Токенизация

Выполним токенизацию.

In [29]:
%%time

tokenizer = transformers.BertTokenizer(vocab_file=f'{bert_path}vocab.txt')
tokenized = data.corp.apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

Wall time: 2min 46s


In [30]:
tokenized.values[:2]

array([list([101, 7108, 1135, 646, 11179, 212, 1189, 1120, 743, 23467, 16883, 13256, 205, 5442, 1015, 17464, 759, 3920, 112, 189, 3498, 6919, 16762, 769, 8354, 683, 812, 3245, 1122, 178, 4751, 752, 1020, 18163, 4661, 22084, 15808, 662, 1712, 813, 112, 189, 5782, 646, 27821, 816, 646, 1396, 3674, 1290, 178, 112, 182, 2623, 980, 102]),
       list([101, 173, 112, 1123, 229, 728, 2697, 713, 3582, 5922, 178, 112, 182, 9321, 5342, 708, 5438, 1396, 28640, 17484, 15626, 223, 102])],
      dtype=object)

Ознакомимся с конфигурацией модели.

In [31]:
json.load(open(f'{bert_path}bert_config.json', 'r'))

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 28996}

Конфигурация модели включает в себя ограничение на максимальную длину эмбеддинга (параметр *max_position_embeddings*).

Найдём максимальную длину токена (соответствует длине эмбеддинга) в нашем датасете.

In [32]:
len(max(tokenized, key=len))

2145

Для корректной работы модели необходимо оставить в датасете только те объекты, которые не превышают установленное ограничение.

В целях экономии машинного времени ограничим максимальную длину эмбеддинга в ещё большей степени.

In [33]:
max_position_embeddings = 80

tokenized_short = tokenized.apply(position_embeddings_filter, max_pos=max_position_embeddings)
tokenized_short.drop(index=tokenized_short[tokenized_short == 0].index, inplace=True)

print('amount of the items:\noriginal:   {}\nrestricted: {}\nfraction:   {:.1%}'
      .format(tokenized.shape[0],
              tokenized_short.shape[0],
              tokenized_short.shape[0] / tokenized.shape[0])
     )

amount of the items:
original:   159571
restricted: 114393
fraction:   71.7%


Данному условию (ограничению по длине) удовлетворяет большая часть датасета, что хорошо с точки зрения репрезентативности:  большинство текстов - действительно не очень длинные.

Однако, с точки зрения машинного времени (доступной вычислительной мощности) придётся ограничиться лишь небольшой частью вновь полученного набора данных.

Сформируем выборку, сохранив баланс классов.

In [34]:
tokenized_n_samples = 10000

tokenized_short_sample_index = sample_func(data.loc[tokenized_short.index],
                                           tokenized_n_samples,
                                           'target',
                                           frac_one=frac_class_1,
                                           return_index=True)

tokenized_short_sample = tokenized_short[tokenized_short_sample_index]

tokenized_short_sample.reset_index(drop=True, inplace=True)
tokenized_short_sample.shape

(10000,)

Сделаем длину всех токенов одинаковой, равной длине самого длинного токена. Для этого удлиним массивы до необходимой длины. Добавленным элементам присвоим нулевые значения.

In [35]:
%%time

padded = []
for i in notebook.tqdm(range(tokenized_short_sample.shape[0])):
    padded.append(tokenized_short_sample.values[i] + [0]*(max_position_embeddings-len(tokenized_short_sample.values[i])))


Wall time: 312 ms


In [36]:
%%time

padded = np.array(padded)
padded.shape

Wall time: 77 ms


(10000, 80)

In [37]:
padded[:2]

array([[  101,   776,  7640, 21805,   980,   178,   854,  5438,   688,
          646,  3189,   102,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0],
       [  101,   788,  3075,  3510,   734,   788,  3075,  3510,   734,
          738,  2145,  3520,   182,   203,   233, 15814,  5540,  3277,
          646,  2067,  4206, 28201,   689,  9294,   662,  1238,  3739,
          102,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            

При помощи маски *attention_mask* обозначим ненулевые позиции в *padded* единицами, а нулевые (незначащие) - нулями.

In [38]:
%%time

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

Wall time: 3 ms


(10000, 80)

In [39]:
attention_mask[:2]

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

#### Формирование эмбеддингов

Инициализируем конфигурацию модели и саму модель.

In [40]:
bert_config = transformers.BertConfig.from_json_file(f'{bert_path}bert_config.json')
bert_model = transformers.BertModel.from_pretrained(f'{bert_path}{bert_model_file}', config=bert_config)

Выполним векторизацию - сформируем эмбеддинги.

In [41]:
%%time

batch_size = 200
embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
    padded_batch = torch.LongTensor(padded[(batch_size * i):(batch_size*(i+1))] )
    attention_mask_batch = torch.LongTensor(attention_mask[(batch_size * i):(batch_size*(i+1))] )
    
    with torch.no_grad():
        embeddings_batch = bert_model(input_ids=padded_batch, attention_mask=attention_mask_batch)
    
    embeddings.append(embeddings_batch[0][:, 0, :].numpy())


Wall time: 21min 59s


Объединим эмбеддинги в матрицу признаков.

In [42]:
bert_features = np.concatenate(embeddings)
bert_features.shape

(10000, 768)

In [43]:
bert_features[:2]

array([[ 0.49360603, -0.23898077,  0.4389935 , ...,  0.6176871 ,
        -0.86993784,  0.13711601],
       [ 0.8886719 , -0.20079303,  0.1106492 , ...,  0.46675897,
        -0.6267636 , -0.28431734]], dtype=float32)

#### Адаптация массива эмбеддингов к применяемым функциям

Для корректной работы используемых в работе функций необходимо поместить массив эмбеддингов вместе с массивом целевого признака в датафрейм.

Убедимся, что длины массивов совпадают.

In [44]:
bert_features.shape[0] == data.target[tokenized_short_sample_index].shape[0]

True

Сформируем датафрейм.

In [45]:
data_bert = pd.DataFrame(data=bert_features)

columns_emb = {}
for i in range(len(data_bert.columns)):
    columns_emb[i] = f'emb_{i}'

data_bert.rename(columns_emb, axis=1, inplace=True)
data_bert_features = data_bert.columns

data_bert = data_bert.join(data.target[tokenized_short_sample_index].reset_index(drop=True),
                           on=data_bert.index,
                           how='left',
                          )
data_bert.head()

,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,...,emb_759,emb_760,emb_761,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,target
0,0.493606,-0.238981,0.438994,0.130449,-0.478443,0.161358,-0.200543,0.097896,0.710276,-0.234599,...,0.402100,0.744939,0.365976,0.059414,-0.111625,-0.125937,0.617687,-0.869938,0.137116,0
1,0.888672,-0.200793,0.110649,0.001981,-0.390708,0.412642,-0.414981,0.017419,0.302613,-0.231825,...,0.310539,0.429558,-0.017159,-0.316017,-0.394899,0.065953,0.466759,-0.626764,-0.284317,1
2,0.362144,0.079172,0.131711,0.040333,-0.674066,0.578170,0.437802,0.174256,0.246273,0.085462,...,0.393744,0.113762,0.127942,0.153627,0.025253,-0.184254,0.365620,-0.005807,0.364162,0
3,0.401402,-0.284934,-0.235779,-0.091475,-0.338438,0.213594,0.201409,-0.110042,0.350604,-0.260918,...,0.586635,0.344407,-0.213413,-0.123215,0.112594,-0.273448,0.261232,-0.189423,0.407664,1
4,0.613844,-0.016272,0.348877,0.026169,-0.656297,0.383256,-0.117549,0.246334,0.531314,0.030343,...,0.621736,0.235166,-0.045918,0.249991,-0.340269,0.197663,0.338269,-0.749350,-0.273718,0


## Формирование выборок

Сформируем обучающую, валидационную и тестовую выборки.

Поскольку исходная выборка содержит весьма большое количество объектов (150k+), размеры валидационной и тестовой выборок можно взять несколько меньше, чем обычно (в пользу обучающей выборки): разделение выполним в пропорции 4:1:1.

Признак и целевой признак:
* признак - векторизированный корпус текстов;
* целевой признак - столбец с классом.

### train_valid | test

Разделим исходную выборку на обучающую и тестовую. Разделять будем, формируя массивы с индексом. Затем уже по индексу будем брать из датафрейма непосредственно сами данные.

При разделении сохраним баланс классов: при помощи функции *class_ratio_control* соотношение классов в исходной, обучающей и тестовой выборках будет различаться не более, чем на заданный процент. По умолчанию установлено 5%, т.е. соответствие - не сверхстрогое, а такое, какое может встретиться в реальных данных на практике. При этом исключены случайные сильные перекосы.

In [46]:
data_index_train_valid, data_index_test = class_ratio_control(df=data)

ratio of the classes:
original: 0.1132
train:    0.1137
test:     0.1104



In [47]:
d(len(data_index_train_valid))
d(len(data_index_test))

132975

26596

In [48]:
data_index_train_valid_bert, data_index_test_bert = class_ratio_control(df=data_bert)

ratio of the classes:
original: 0.1131
train:    0.1130
test:     0.1136



In [49]:
d(len(data_index_train_valid_bert))
d(len(data_index_test_bert))

8333

1667

### cv: train | valid

Кросс-валидацию развернём вручную, так как необходимо учесть две особенности:
* и на входе, и на выходе - массивы с индексами объектов, а не с непосредственно значениями; библиотечные функции больше ориентированы на работу с массивами значений, и вворачивать туда индексы - довольно громоздко;
* при малой величине *cv* (малом числе проходов кросс-валидации) выборка должна разбиваться на тестовую и валидационную в соответствии с заданным размером последней.

Второе условие необходимо для того, чтобы больше данных отводилось на обучающую выборку. К примеру, при *cv*=3 выборка обычно разбивается в пропорции *train*=67%, *valid*=33%. В нашем случае размер валидационной выборки указывается явно, и, таким образом, при малом *cv* и *valid_size* (точнее, при *cv * valid_size < 1.0*) не все данные побывают в валидационной выборке. Зато обучение при каждом проходе будет выполняться на большем объёме данных.

Пример работы функции *cv_indexes*.

In [50]:
# defaults: cv=3, valid_size=0.2

tr, vl = cv_indexes(np.arange(10, 101, 10))

for i in range(3):
    print(f'train_{i}: {tr[i]} valid_{i}: {vl[i]}')

train_0: [ 30  40  50  60  70  80  90 100] valid_0: [10 20]
train_1: [ 10  20  50  60  70  80  90 100] valid_1: [30 40]
train_2: [ 10  20  30  40  70  80  90 100] valid_2: [50 60]


# 2. Обучение

## Функции

In [51]:
# The function transforms the dict with hyperparameters into the dataframe format

def dict_to_df(params_dict={}):
    return pd.DataFrame(data=itertools.product(*params_dict.values()), columns=params_dict.keys())

In [52]:
# The function transforms features into the necessary format

def features_processing(data, algorithm, purpose, counter=None):
    if algorithm == 'tf_idf':
        if purpose == 'train':
            return counter.fit_transform(data)
        elif (purpose == 'valid') or (purpose == 'test'):
            return counter.transform(data)
        else:
            return data

    if algorithm == 'tf_idf_catboost':
        if purpose == 'train':
            return np.array(pd.DataFrame.sparse.from_spmatrix(counter.fit_transform(data)))
        elif (purpose == 'valid') or (purpose == 'test'):
            return np.array(pd.DataFrame.sparse.from_spmatrix(counter.transform(data)))
        else:
            return data

    elif algorithm == 'bert':
        if purpose == 'train':
            return data
        elif (purpose == 'valid') or (purpose == 'test'):
            return data
        else:
            return data
    
    else:
        return data

In [53]:
# The function computes the models using the hyperparameters grid, implements cross-validation,
# returns the results of the best model: F1-score and the values of the hyperparameters

def custom_grid_search_cv(model_init,
                          df_params,
                          features_col,
                          features_algorithm,
                          counter=None,
                          df=data,
                          index_train_valid=data_index_train_valid,
                          cv=3,
                          valid_size=0.2
                         ):
    
    index_train, index_valid = cv_indexes(index_train_valid, cv=cv, valid_size=valid_size)
    df_score = df_params.copy()
    full_params_dict = []
    
    for i in notebook.tqdm(range(df_params.shape[0])):
        model = None
        try:
            model = model_init.copy()
        except:
            model = model_init
        
        model.set_params(**df_params.loc[i].to_dict())

        score = []
        exception = False
        for j in range(cv):
            X_train = features_processing(data=df.loc[index_train[j], features_col],
                                          algorithm=features_algorithm,
                                          purpose='train',
                                          counter=counter
                                         )
            X_valid = features_processing(data=df.loc[index_valid[j], features_col],
                                          algorithm=features_algorithm,
                                          purpose='valid',
                                          counter=counter
                                         )
            y_train = df.loc[index_train[j], 'target']
            y_valid = df.loc[index_valid[j], 'target']
            
            try:
                model.fit(X_train, y_train)
                y_pred = model.predict(X_valid)
                score.append(f1_score(y_valid, y_pred))
            except:
                exception = True
                break

        if exception == False:
            df_score.loc[i, 'score'] = np.mean(score)
            print(df_score.loc[i].to_dict())
        else:
            df_score.loc[i, 'score'] = 0.0
        
        full_params_dict.append(model.get_params())
        
    best_score = df_score.loc[df_score.score.idxmax(), 'score']
    best_params = df_score.loc[df_score.score.idxmax(), df_score.columns[:-1]].to_dict()
    full_best_params = full_params_dict[df_score.score.idxmax()]

    print('the best score:  {:.4f}\nthe best parameters: {}'.format(best_score, best_params))

    return best_score, best_params, full_best_params

In [54]:
# The function tests the model on the test sample

def testing(model,
            params_dict,
            features_col,
            features_algorithm,
            counter=None,
            df=data,
            index_train=data_index_train_valid,
            index_test=data_index_test
           ):
    
    X_train = features_processing(data=df.loc[index_train, features_col],
                                  algorithm=features_algorithm,
                                  purpose='train',
                                  counter=counter
                                 )
    X_test  = features_processing(data=df.loc[index_test, features_col],
                                  algorithm=features_algorithm,
                                  purpose='test',
                                  counter=counter
                                 )
    y_train = df.loc[index_train, 'target']
    y_test  = df.loc[index_test, 'target']
    
    model.set_params(**params_dict)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = f1_score(y_test, y_pred)

    print('testing score:  {:.4f}'.format(score))

    return score

## Обучение и валидация (*TF-IDF*)

### Random Forest

Инициализируем модель.

In [55]:
model_rf = RandomForestClassifier(n_estimators=100,
                                  n_jobs=-1,
                                  random_state=r_state,
                                  class_weight='balanced'
                                 )

Создадим словарь - сетку гиперпараметров.

В процессе исследования были опробованы значения, различающиеся на порядки. Также были опробованы различные значения *n_estimators* (от 20 до 200). В финальной (текущей) версии - диапазоны, включающие в себя "экстремумы": оптимальные значения не являются границами диапазонов, а находятся внутри них.

In [56]:
params_rf = {
    'max_depth': np.geomspace(2**10, 2**11, 3, dtype='int'),
    'min_samples_split': np.geomspace(2**8, 2**9, 3, dtype='int')
}

params_rf

{'max_depth': array([1024, 1448, 2048]),
 'min_samples_split': array([256, 362, 512])}

Выполним обучение.

In [57]:
%%time

best_score_rf, \
best_params_rf, \
full_best_params_rf \
= custom_grid_search_cv(model_init=model_rf,
                        df_params=dict_to_df(params_rf),
                        features_col='corp_lemm',
                        features_algorithm='tf_idf',
                        counter=count_tf_idf
                       )

{'max_depth': 1024.0, 'min_samples_split': 256.0, 'score': 0.711525863087545}
{'max_depth': 1024.0, 'min_samples_split': 362.0, 'score': 0.7119640997804071}
{'max_depth': 1024.0, 'min_samples_split': 512.0, 'score': 0.7099752570428914}
{'max_depth': 1448.0, 'min_samples_split': 256.0, 'score': 0.7105043683969968}
{'max_depth': 1448.0, 'min_samples_split': 362.0, 'score': 0.7117722327744791}
{'max_depth': 1448.0, 'min_samples_split': 512.0, 'score': 0.7128411056487923}
{'max_depth': 2048.0, 'min_samples_split': 256.0, 'score': 0.7098645882873761}
{'max_depth': 2048.0, 'min_samples_split': 362.0, 'score': 0.7104902489569543}
{'max_depth': 2048.0, 'min_samples_split': 512.0, 'score': 0.712756914180244}

the best score:  0.7128
the best parameters: {'max_depth': 1448.0, 'min_samples_split': 512.0}
Wall time: 14min 57s


### Logistic Regression

Инициализируем модель.

In [58]:
model_lr = LogisticRegression(n_jobs=-1,
                              random_state=r_state,
                              class_weight='balanced',
                              multi_class='ovr'
                             )

Создадим словарь - сетку гиперпараметров.

Алгоритм оптимизации *saga* использовать не будем: он даёт весьма средние результаты, но при этом в некоторых случаях приводит к зависанию ядра (или - если это не зависание - выполняет расчёт неприемлемо медленно). Несовместимые значения будут автоматически пропущены.

In [59]:
params_lr = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag'],
    'penalty': ['l1', 'l2']
}

params_lr

{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag'], 'penalty': ['l1', 'l2']}

Выполним обучение.

In [60]:
%%time

best_score_lr, \
best_params_lr, \
full_best_params_lr \
= custom_grid_search_cv(model_init=model_lr,
                        df_params=dict_to_df(params_lr),
                        features_col='corp_lemm',
                        features_algorithm='tf_idf',
                        counter=count_tf_idf
                       )

{'solver': 'newton-cg', 'penalty': 'l2', 'score': 0.7426195440135599}
{'solver': 'lbfgs', 'penalty': 'l2', 'score': 0.742579885582794}
{'solver': 'liblinear', 'penalty': 'l1', 'score': 0.7434814589730013}
{'solver': 'liblinear', 'penalty': 'l2', 'score': 0.7425503583176756}
{'solver': 'sag', 'penalty': 'l2', 'score': 0.7426195440135599}

the best score:  0.7435
the best parameters: {'solver': 'liblinear', 'penalty': 'l1'}
Wall time: 2min 43s


### Light GBM

#### Описание процесса подбора гиперпараметров

Здесь пошагово описан процесс подбора гиперпараметров. Привести **непосредственно** сами расчёты (промежуточные) не представляется возможным ввиду огромных затрат машинного времени на каждый перезапуск тетрадки. Приведён только **окончательный** вариант. Развёрнутые **результаты** расчёта приведены в Приложении в конце работы.

**Шаг 1**. В процессе исследования были опробованы различные комбинации значений в весьма широких пределах (см. ниже) - расчёт длился на протяжении 10-12 часов. Это - не считая пробных предварительных расчётов для определения порядка значений в первом приближении. Анализ полученных результатов выявил, каким образом влияет на метрику каждый из гиперпараметров, и это позволило исключить из сетки заведомо неоптимальные значения. Тем самым время обучения было сокращено до приемлемого.

* n_estimators: 100,
* num_iterations: 400,
* learning_rate: 0.05
* num_leaves: (32,  45,  63,  90), //*90 - не завершён*
* min_data_in_leaf: (2, 3, 7),
* max_depth: (16, 22, 31, 45, 63)

**Шаг 2**. Глубина дерева *max_depth* в алгоритме *Light GBM* не настолько влияет на результат, как, например, в *Random Forest*. Чтобы сократить машинное время, не будем перебирать этот параметр, а присвоим ему фиксированное значение. Оптимальным представляется *max_depth*=32.

Минимальное количество строк в листе *min_data_in_leaf* - в данном случае чем меньше, тем лучше. Поэтому будем рассматривать два значения: *min_data_in_leaf*=\[2, 4\].

Число листьев *num_leaves* - чем больше, тем лучше. Но увеличение может привести к переобучению модели. Поэтому рассмотрим только три значения из опробованных ранее: *num_leaves*=\[32, 45, 63\].

Также попробуем загрубить бустинг: число итераций уменьшим вдвое (200 вместо 400), но при этом увеличим шаг (0.10 вместо 0.05).

* n_estimators: 100,
* num_iterations: 200,
* learning_rate: 0.10
* num_leaves: (32,  45,  63)
* min_data_in_leaf: (2, 4),
* max_depth: 32

**Шаг 3**. Результат получился, хоть и незначительно, но хуже. Это означает, что следует попробовать пойти по пути увеличения числа итераций и уменьшения шага спуска.

Так как при этом кратно увеличится время расчёта, придётся отказаться от перебора тех параметров, что мы подбирали ранее. Примем такие фиксированные их значения, которые давали лучшие результаты.

Примем *min_data_in_leaf*=2; *num_leaves*=64. Будем иметь в виду, что в случае низких результатов на тестовой выборке надо попробовать уменьшить *num_leaves* (попробовать значения 32, 45), так как причиной может быть переобучение.

* n_estimators: 150,
* num_iterations: 500,
* learning_rate: 0.05
* num_leaves: 64
* min_data_in_leaf: 2,
* max_depth: 32

**Шаг 4**. Результат получился немного лучше, но расчёт длится слишком долго. Хотелось бы относительно безболезненно сократить машинное время. Можно попробовать уменьшить число итераций кросс-валидации (2 вместо 3) - на столь большой выборке качество может практически не снизиться. Также попробуем уменьшить количество корзин *max_bin* (64 вместо 255). Уменьшение *max_bin* в общем случае снижает точность, что - плохо, но при этом - уменьшает эффект переобучения, что - хорошо.

Результат шага 4 оказался удовлетворительным.

#### Построение модели

Инициализируем модель.

In [61]:
model_gb = lgb.LGBMClassifier(n_estimators=150,
                              n_jobs=-1,
                              random_state=r_state,
                              class_weight='balanced',
                              objective='binary',
                              metric='binary_logloss',
                              num_iterations=500,
                              learning_rate=0.05,
                              boosting='gbdt',
                              max_depth=32,
                              max_bin=64,
                              num_leaves=64,
                              min_data_in_leaf=2
                             )

Выполним обучение.

In [62]:
%%time

best_score_gb, \
best_params_gb, \
full_best_params_gb \
= custom_grid_search_cv(model_init=model_gb,
                        df_params=dict_to_df(),
                        features_col='corp_lemm',
                        features_algorithm='tf_idf',
                        counter=count_tf_idf,
                        cv=2
                       )

{'score': 0.7729056585818308}

the best score:  0.7729
the best parameters: {}
Wall time: 11min 35s


### CatBoost

#### Формирование уменьшенного датасета

*CatBoost* не работает с разреженными матрицами (sparse matrix). Можно привести массив признаков к обычной матрице, но тогда его объём составит 97.6 Гб, что не приемлемо с точки зрения доступного аппаратного обеспечения.

Чтобы опробовать работу алгоритма *CatBoost*, сформируем уменьшенный датасет из исходного. Элементы выберем случайным образом.

Сформируем уменьшенный датасет и массивы индексов к нему. При этом сохраним баланс классов, как в исходном датасете.

In [63]:
n_samples_cb = 12000

data_cb = sample_func(data, n_samples_cb, 'target', frac_one=frac_class_1)
data_cb.shape

(12000, 3)

In [64]:
data_index_train_valid_cb, data_index_test_cb = class_ratio_control(df=data_cb)

ratio of the classes:
original: 0.1132
train:    0.1141
test:     0.1086



#### Описание процесса подбора гиперпараметров

Описание процесса подбора - аналогично *LightGBM*.

**Шаг 1**. В качестве бэйслайна установлены значения, аналогичные подобранным в *LightGBM* (так как эти алгоритмы похожи между собой). Результат получился удовлетворительным, но всё равно требовал дальнейшего улучшения: в зависимости от состава обучающей выборки (формирование обучающей и тестовой выборок не закреплено параметром *random_state* - этого, в свою очередь, требует алгоритм разбиения на train/test) результат на тестовой выборке получался то выше, то ниже, чем требование ТЗ (*F1-score* > 0.75).

Необходимо отметить, что влияние состава выборки на результат оказалось даже сильнее, чем изменение значений гиперпараметров (тех, что перебирались по сетке). Причина: тексты сильно различаются между собой по длине и по составу слов, а из-за малого размера выборки такие различия не "усредняются" и сказываются намного заметнее, чем на исходной большой выборке. Такое влияние - непредсказуемое, и влияет на метрику весьма существенно.

* loss_function: 'Logloss',
* iterations: 500,
* learning_rate: 0.05,
* grow_policy: 'Depthwise',
* min_data_in_leaf: 2,
* auto_class_weights: 'Balanced',
* thread_count: -1,
* logging_level: 'Silent'
* depth: (4, 7, 16),
* border_count: (61, 125, 254)

**Шаг 2**. Расчёт длится достаточно долго. Попробуем загрубить спуск, а глубину *depth* - переберём немного более дробно.

* loss_function: 'Logloss',
* iterations: 200,
* learning_rate: 0.10,
* grow_policy: 'Depthwise',
* min_data_in_leaf: 2,
* auto_class_weights: 'Balanced',
* thread_count: -1,
* logging_level: 'Silent'
* depth: (4, 6, 10, 16),
* border_count: (61, 125, 254)

**Шаг 3**. Загрубление спуска не оказало негативного влияния на обучение: моделям, чтобы завершить обучение, хватает как количества шагов, так и их дискретности. Время обучения хотелось бы сократить ещё, и такая возможность есть: необходимо сократить сетку гиперпараметров. В отличие от *LightGBM*, от неё не получится отказаться совсем: из-за малого размера датасета результаты слишком нестабильны - не понятно, какие значения окажутся в итоге лучше. Но в этом и нет необходимости: опять же, из-за размера датасета небольшой перебор всё-таки можно себе позволить с точки зрения затрат машинного времени. Уберём малые значения *depth*: с ними результат оказывается в среднем хуже. Уберём и самое малое значение *border_count*: иногда именно при нём получается самый лучший результат, но чаще - всё-таки при больших значениях.

* loss_function: 'Logloss',
* iterations: 200,
* learning_rate: 0.10,
* grow_policy: 'Depthwise',
* min_data_in_leaf: 2,
* auto_class_weights: 'Balanced',
* thread_count: -1,
* logging_level: 'Silent'
* depth: (10, 16),
* border_count: (125, 254)

Результат шага 3 оказался условно\* удовлетворительным. Окончательный вариант расчёта приведён ниже.

(\*) - результат на тестовой выборке варьируется в относительно широких пределах; бывает как лучше, так и хуже, чем на валидационной выборке; иногда удовлетворяет требованиям ТЗ, но чаще - нет. К сожалению, отсутствует возможность обучить и проверить модель на полном датасете.

#### Построение модели

Инициализируем модель.

Установим значения, аналогичные установленным в *LightGBM*.

In [65]:
model_cb = cb.CatBoostClassifier(loss_function='Logloss',
                                 iterations=200,
                                 learning_rate=0.10,
                                 grow_policy='Depthwise',
                                 min_data_in_leaf=2,
                                 auto_class_weights='Balanced',
                                 thread_count=-1,
                                 logging_level='Silent'
                                )

Создадим словарь - сетку гиперпараметров.

In [66]:
params_cb = {
    'depth': [10, 16],
    'border_count': np.geomspace(2**7, 2**8, 2, dtype='int') - 2
}

params_cb

{'depth': [10, 16], 'border_count': array([125, 254])}

Выполним обучение.

In [67]:
%%time

best_score_cb, \
best_params_cb, \
full_best_params_cb \
= custom_grid_search_cv(model_init=model_cb,
                        df_params=dict_to_df(params_cb),
                        features_col='corp_lemm',
                        features_algorithm='tf_idf_catboost',
                        counter=count_tf_idf,
                        cv=2,
                        df=data_cb,
                        index_train_valid=data_index_train_valid_cb
                       )

{'depth': 10.0, 'border_count': 125.0, 'score': 0.6981474665685192}
{'depth': 10.0, 'border_count': 254.0, 'score': 0.7057823129251701}
{'depth': 16.0, 'border_count': 125.0, 'score': 0.709229176620481}
{'depth': 16.0, 'border_count': 254.0, 'score': 0.724822437937192}

the best score:  0.7248
the best parameters: {'depth': 16.0, 'border_count': 254.0}
Wall time: 14min 57s


## Тестирование (*TF-IDF*)

Выполним тестирование моделей, показавших лучшие результаты на валидационной выборке.

In [68]:
%%time

test_score_rf = testing(model=RandomForestClassifier(),
                        params_dict=full_best_params_rf,
                        features_col='corp_lemm',
                        features_algorithm='tf_idf',
                        counter=count_tf_idf
                       )

testing score:  0.7130
Wall time: 43.6 s


In [69]:
%%time

test_score_lr = testing(model=LogisticRegression(),
                        params_dict=full_best_params_lr,
                        features_col='corp_lemm',
                        features_algorithm='tf_idf',
                        counter=count_tf_idf
                       )

testing score:  0.7386
Wall time: 8.34 s


In [70]:
%%time

test_score_gb = testing(model=lgb.LGBMClassifier(),
                        params_dict=full_best_params_gb,
                        features_col='corp_lemm',
                        features_algorithm='tf_idf',
                        counter=count_tf_idf
                       )

testing score:  0.7667
Wall time: 6min 22s


In [71]:
%%time

test_score_cb = testing(model=cb.CatBoostClassifier(),
                        params_dict=full_best_params_cb,
                        features_col='corp_lemm',
                        features_algorithm='tf_idf_catboost',
                        counter=count_tf_idf,
                        df=data_cb,
                        index_train=data_index_train_valid_cb,
                        index_test=data_index_test_cb
                       )

testing score:  0.7348
Wall time: 3min 7s


## Обучение и валидация (*BERT*)

Алгоритм и логика действий в целом - такие же, как и в разделе выше.

### Random Forest

Инициализируем модель.

In [72]:
model_rf_bert = RandomForestClassifier(n_estimators=700,
                                       n_jobs=-1,
                                       random_state=r_state,
                                       class_weight='balanced',
                                       max_depth=8
                                      )

Создадим словарь - сетку гиперпараметров.

In [73]:
params_rf_bert = {
    'min_samples_split': np.geomspace(2**5, 2**7, 4, dtype='int')
}

params_rf_bert

{'min_samples_split': array([ 32,  50,  80, 127])}

Выполним обучение.

In [74]:
%%time

best_score_rf_bert, \
best_params_rf_bert, \
full_best_params_rf_bert \
= custom_grid_search_cv(model_init=model_rf_bert,
                        df_params=dict_to_df(params_rf_bert),
                        features_col=data_bert_features,
                        features_algorithm='bert',
                        df=data_bert,
                        index_train_valid=data_index_train_valid_bert
                       )

{'min_samples_split': 32.0, 'score': 0.6368787731016833}
{'min_samples_split': 50.0, 'score': 0.6441596909030362}
{'min_samples_split': 80.0, 'score': 0.6356582806105338}
{'min_samples_split': 127.0, 'score': 0.6246159899514218}

the best score:  0.6442
the best parameters: {'min_samples_split': 50.0}
Wall time: 2min 11s


### Logistic Regression

Инициализируем модель.

In [75]:
model_lr_bert = LogisticRegression(n_jobs=-1,
                                   random_state=r_state,
                                   class_weight='balanced',
                                   multi_class='ovr'
                                  )

Создадим словарь - сетку гиперпараметров.

In [76]:
params_lr_bert = {
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag'],
    'penalty': ['l1', 'l2']
}

params_lr_bert

{'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag'], 'penalty': ['l1', 'l2']}

Выполним обучение.

In [77]:
%%time

best_score_lr_bert, \
best_params_lr_bert, \
full_best_params_lr_bert \
= custom_grid_search_cv(model_init=model_lr_bert,
                        df_params=dict_to_df(params_lr_bert),
                        features_col=data_bert_features,
                        features_algorithm='bert',
                        df=data_bert,
                        index_train_valid=data_index_train_valid_bert
                       )

{'solver': 'newton-cg', 'penalty': 'l2', 'score': 0.6737860303564983}
{'solver': 'lbfgs', 'penalty': 'l2', 'score': 0.6698397740752227}
{'solver': 'liblinear', 'penalty': 'l1', 'score': 0.6692406697883831}
{'solver': 'liblinear', 'penalty': 'l2', 'score': 0.6737134317079843}
{'solver': 'sag', 'penalty': 'l2', 'score': 0.6578106116520362}

the best score:  0.6738
the best parameters: {'solver': 'newton-cg', 'penalty': 'l2'}
Wall time: 3min 6s


### Light GBM

Инициализируем модель.

In [78]:
model_gb_bert = lgb.LGBMClassifier(n_estimators=150,
                                   n_jobs=-1,
                                   random_state=r_state,
                                   class_weight='balanced',
                                   objective='binary',
                                   metric='binary_logloss',
                                   num_iterations=500,
                                   learning_rate=0.05,
                                   boosting='gbdt',
                                   max_depth=16,
                                   num_leaves=64,
                                   min_data_in_leaf=2
                                  )

Создадим словарь - сетку гиперпараметров.

In [79]:
params_gb_bert = {
    'max_bin': np.geomspace(2**5, 2**7, 3, dtype='int')
}

params_gb_bert

{'max_bin': array([ 32,  63, 127])}

Выполним обучение.

In [80]:
%%time

best_score_gb_bert, \
best_params_gb_bert, \
full_best_params_gb_bert \
= custom_grid_search_cv(model_init=model_gb_bert,
                        df_params=dict_to_df(params_gb_bert),
                        features_col=data_bert_features,
                        features_algorithm='bert',
                        df=data_bert,
                        index_train_valid=data_index_train_valid_bert,
                        cv=2
                       )

{'max_bin': 32.0, 'score': 0.6306230341966139}
{'max_bin': 63.0, 'score': 0.6339362157534246}
{'max_bin': 127.0, 'score': 0.6250374812593704}

the best score:  0.6339
the best parameters: {'max_bin': 63.0}
Wall time: 3min 47s


### CatBoost

Инициализируем модель.

In [81]:
model_cb_bert = cb.CatBoostClassifier(loss_function='Logloss',
                                      iterations=200,
                                      learning_rate=0.05,
                                      grow_policy='Depthwise',
                                      min_data_in_leaf=2,
                                      auto_class_weights='Balanced',
                                      thread_count=-1,
                                      logging_level='Silent'
                                     )

Создадим словарь - сетку гиперпараметров.

In [82]:
params_cb_bert = {
    'depth': np.geomspace(2**2, 2**3, 3, dtype='int'),
    'border_count': np.geomspace(2**3, 2**5, 3, dtype='int') - 2
}

params_cb_bert

{'depth': array([4, 5, 7]), 'border_count': array([ 5, 14, 30])}

Выполним обучение.

In [83]:
%%time

best_score_cb_bert, \
best_params_cb_bert, \
full_best_params_cb_bert \
= custom_grid_search_cv(model_init=model_cb_bert,
                        df_params=dict_to_df(params_cb_bert),
                        features_col=data_bert_features,
                        features_algorithm='bert',
                        df=data_bert,
                        index_train_valid=data_index_train_valid_bert,
                        cv=2
                       )

{'depth': 4.0, 'border_count': 5.0, 'score': 0.6224557044079515}
{'depth': 4.0, 'border_count': 14.0, 'score': 0.6353321242348674}
{'depth': 4.0, 'border_count': 30.0, 'score': 0.6343151958725778}
{'depth': 5.0, 'border_count': 5.0, 'score': 0.64508547008547}
{'depth': 5.0, 'border_count': 14.0, 'score': 0.6587094096142502}
{'depth': 5.0, 'border_count': 30.0, 'score': 0.6638226281319473}
{'depth': 7.0, 'border_count': 5.0, 'score': 0.6539835828102365}
{'depth': 7.0, 'border_count': 14.0, 'score': 0.6591567908602609}
{'depth': 7.0, 'border_count': 30.0, 'score': 0.6559738479384161}

the best score:  0.6638
the best parameters: {'depth': 5.0, 'border_count': 30.0}
Wall time: 3min 2s


## Тестирование (*BERT*)

Выполним тестирование моделей, показавших лучшие результаты на валидационной выборке.

In [84]:
%%time

test_score_rf_bert = testing(model=RandomForestClassifier(),
                             params_dict=full_best_params_rf_bert,
                             features_col=data_bert_features,
                             features_algorithm='bert',
                             df=data_bert,
                             index_train=data_index_train_valid_bert,
                             index_test=data_index_test_bert
                            )

testing score:  0.6189
Wall time: 14 s


In [85]:
%%time

test_score_lr_bert = testing(model=LogisticRegression(),
                             params_dict=full_best_params_lr_bert,
                             features_col=data_bert_features,
                             features_algorithm='bert',
                             df=data_bert,
                             index_train=data_index_train_valid_bert,
                             index_test=data_index_test_bert
                            )

testing score:  0.6683
Wall time: 36 s


In [86]:
%%time

test_score_gb_bert = testing(model=lgb.LGBMClassifier(),
                             params_dict=full_best_params_gb_bert,
                             features_col=data_bert_features,
                             features_algorithm='bert',
                             df=data_bert,
                             index_train=data_index_train_valid_bert,
                             index_test=data_index_test_bert
                            )

testing score:  0.5824
Wall time: 37.8 s


In [87]:
%%time

test_score_cb_bert = testing(model=cb.CatBoostClassifier(),
                             params_dict=full_best_params_cb_bert,
                             features_col=data_bert_features,
                             features_algorithm='bert',
                             df=data_bert,
                             index_train=data_index_train_valid_bert,
                             index_test=data_index_test_bert
                            )

testing score:  0.6361
Wall time: 9.26 s


## Результаты (значения метрик *F1-score*)

Сведём результаты расчёта (значения метрик *F1-score*) в таблицу.

In [88]:
results = pd.DataFrame(data = [[best_score_rf, best_score_lr, best_score_gb, best_score_cb],
                               [test_score_rf, test_score_lr, test_score_gb, test_score_cb],
                               [best_score_rf_bert, best_score_lr_bert, best_score_gb_bert, best_score_cb_bert],
                               [test_score_rf_bert, test_score_lr_bert, test_score_gb_bert, test_score_cb_bert]
                              ],
                       index = pd.MultiIndex.from_arrays([['TF-IDF', 'TF-IDF', 'BERT', 'BERT'],
                                                          ['validation', 'test', 'validation', 'test']],
                                                         names=('algorithm', 'stage')
                                                        ),
                       columns = ['Random Forest', 'Logistic Regression', 'Light GBM', 'CatBoost']
                      )
results

Random Forest  Logistic Regression  Light GBM  CatBoost
algorithm stage                                                              
TF-IDF    validation       0.712841             0.743481   0.772906  0.724822
          test             0.712970             0.738624   0.766678  0.734807
BERT      validation       0.644160             0.673786   0.633936  0.663823
          test             0.618893             0.668269   0.582375  0.636119

# 3. Выводы

### Подготовка

На этапе подготовки выполнено ознакомление с данными. Выявлен значительный **дисбаланс** классов (1:9). Пропусков нет.

Конечная **цель** подготовки признаков - преобразование текстов в **векторы**: машина понимает только числа. Было рассмотрено два предназначенных для этого алгоритма: ***TF-IDF*** и ***BERT***. Строго говоря, *TF-IDF* - это название не самого алгоритма, а функций, положенных в его основу, но для краткости приемлемо называть его так.

Сформировано **два варианта** корпуса текстов - для каждого из алгоритмов (т.к. есть различия в требованиях к подготовке). Из **обоих** корпусов удалены небуквенные символы, лишние пробелы. Обработка корпуса для ***BERT*** на этом была **завершена** - всё остальное сделает **предобученная** модель. На корпусе для ***TF-IDF*** также **выполнены** удаление стоп-слов, токенизация и лемматизация средствами библиотеки *nltk* (***WordNetLemmatizer***).

В силу **повышенной ресурсоёмкости** процессов, выполняемых моделью *BERT*, корпус текстов для работы с ней пришлось **уменьшить**. После выполнения токенизации **удалены** объекты длиной **более 80 токенов** (осталось 72% объектов), а затем из оставшихся **отобрано** случайным образом **10000** объектов. При этом соотношение классов целевого признака **сохранено** таким же, как в исходном датасете.

Выполнено **разделение** данных (полного датасета, датасета для *BERT*) на тренировочную и тестовую выборки в соотношении 1:6.

### Обучение, валидация, тестирование

Для обучения применялись следующие **алгоритмы**:

* *Random Forest*;
* *Logistic Regression*;
* *Light GBM*;
* *CatBoost*.

**Кросс-валидация** выполнялась в три (*Random Forest*, *Logistic Regression*) либо в два (*Light GBM*, *CatBoost*) прохода. **Особенность** применённого алгоритма кросс-валидации заключается в том, что, возможно, **не все** данные обучающей выборки побывают в валидационной. Устанавливается размер валидационной выборки (доля от размера всей обучающей выборки) и число проходов. Так, если размер равен 0.2, а число проходов - 3, то в валидационной выборке побывает только 60% данных обучающей выборки. Это сделано для **максимизации** объёма данных, на которых модель непосредственно **обучается**.

**Гиперпараметры** подбирались поэтапно: вначале "нащупывался" рабочий диапазон (приведено только описание, результаты - в Приложении), а затем - тонкая настройка (приведён расчёт). Характер рассматриваемого датасета таков, что **процесс обучения** на нём занимает весьма **значительное время** в контексте сроков завершения исследования и доступной вычислительной мощности. Это - ограничения. Поэтому вполне возможно, что достигнутые значения метрик - не предел. Вместе с тем, следует отметить, что оптимизация **признавалась завершённой** только в тот момент, когда дальнейшее изменение гиперпараметров уже **не давало** заметного эффекта. Запас по оптимизации, возможно, не исчерпан в области подготовки признаков: процесс - творческий, и там есть, с чем поэкспериментировать.

**Лучшая модель**, полученная в данном исследовании, создана алгоритмом ***Light GBM*** и обучена на **полном** наборе данных, преобразованных в векторы алгоритмом ***TF-IDF*** (*TfidfVectorizer* библиотеки *Scikit-learn*).  
Кроме того, эта модель - единственная, **стабильно удовлетворяющая** требованиям ТЗ (*F1-score* > 0.75) как на тестовых данных (*F1-score* = **0.76...0.78**), так и на обучающих (*F1-score* = 0.75...0.77).

**На втором месте** - модель, созданная алгоритмом ***Logistic Regression***, обученная на **полном** наборе данных (***TF-IDF***). Её качество - пограничное с требованиями ТЗ: в зависимости от состава выборок (см. ниже - о *random_state*) иногда **удаётся** уложиться в требования ТЗ на тестовой выборке, но это - **нестабильный** результат.

**Остальные** модели **не удовлетворяют** требованиям ТЗ.

Компромисс, связанный с **ограничениями по ресурсам**, отразился на результатах. Ожидалось, что более "умное" преобразование в векторы нейронной сетью ***BERT*** позволит достичь наилучших результатов. Но этого не произошло из-за **отсутствия возможности** обучать модели на полном наборе данных: пришлось обойтись лишь небольшой выборкой (**около 6%** от исходной). В итоге результаты получились значительно **ниже**, чем у моделей, обученных на полном наборе данных и подготовленных более "простым" способом (*TF-IDF*).

**Схожая проблема** возникла с алгоритмом ***CatBoost*** на данных, преобразованных ***TF-IDF*** (пришлось работать с небольшой выборкой). Но первоначальная причина проблемы - в другом: *CatBoost* **не работает** с **разреженными** матрицами, а обычная матрица, полученная из разреженной, **превышает** имеющийся объём ОЗУ на 1-2 порядка. Впрочем, алгоритм работает весьма медленно, и даже на маленькой выборке требуется время, сопоставимое с временем работы других алгоритмов на полном наборе данных. Возможно, это связано с тем, что подаваемая на вход матрица - не разреженная.

Стоит обратить внимание, что модели, обученные на **полном** наборе данных (*Random Forest*, *Logistic Regression*, *Light GBM*, данные преобразованы *TF-IDF*), на **тестовой** выборке показывают результат, как правило, **близкий и немного лучший**, чем на валидационной выборке. Это говорит о следующем:

* модели обучены очень хорошо: нет ни недообучения, ни переобучения;
* объём данных - достаточно большой, чтобы тестовая выборка статистически не отличалась от валидационной;
* улучшение результата на тестовой выборке наблюдается потому, что на фоне первых двух факторов модель во время теста обучается на большей по размеру выборке (тренировочная плюс валидационная).

**О *random_state***. Параметр *random_state* **не зафиксирован** при разбиении на тренировочную и тестовую выборки в целях обеспечения возможности **автоматического подбора** такого разбиения, при котором с заданной точностью **сохраняется соотношение** классов целевого признака.

### Заключение

Единственная модель, которая строго и стабильно удовлетворяет требованиям ТЗ, - модель ***Light GBM***, для обучения которой использовался **полный** набор данных, преобразованных алгоритмом ***TF-IDF*** (*F1-score* = 0.76...0.78).

Также представляет интерес модель ***Logistic Regression***, обученная на **том же** наборе данных. Отставание от необходимого уровня точности у этой модели - **минимальное** (\~0.01), но зато она обладает другим преимуществом - **высокой скоростью** обучения. На тестовых данных модель *Logistic Regression* обучилась и сделала предсказание за 18 секунд против 7 минут у модели *Light GBM*. Это - более чем **в 20 раз быстрее**.

Модель ***Light GBM*** **рекомендуется** к внедрению.

Модель ***Logistic Regression*** с некоторыми оговорками **может быть интересна** для внедрения.

# Приложение

## Результаты подбора гиперпараметров для *LightGBM*  и *CatBoost* (*TF-IDF*)

### *LightGBM*

1)  
n_estimators=100  
n_jobs=-1,  
random_state=r_state,  
class_weight='balanced',  
objective='binary',  
metric='binary_logloss',  
num_iterations=400  
learning_rate=0.05  
{'num_leaves': 32.0, 'min_data_in_leaf': 2.0, 'max_depth': 16.0, 'score': 0.7504880760034182}  
{'num_leaves': 32.0, 'min_data_in_leaf': 2.0, 'max_depth': 22.0, 'score': 0.7518492081820263}  
{'num_leaves': 32.0, 'min_data_in_leaf': 2.0, 'max_depth': 31.0, 'score': 0.7489382456019079}  
{'num_leaves': 32.0, 'min_data_in_leaf': 2.0, 'max_depth': 45.0, 'score': 0.7489382456019079}  
{'num_leaves': 32.0, 'min_data_in_leaf': 2.0, 'max_depth': 63.0, 'score': 0.7489382456019079}  
{'num_leaves': 32.0, 'min_data_in_leaf': 3.0, 'max_depth': 16.0, 'score': 0.7516566670198676}  
{'num_leaves': 32.0, 'min_data_in_leaf': 3.0, 'max_depth': 22.0, 'score': 0.7526529573831803}  
{'num_leaves': 32.0, 'min_data_in_leaf': 3.0, 'max_depth': 31.0, 'score': 0.7495301335789781}  
{'num_leaves': 32.0, 'min_data_in_leaf': 3.0, 'max_depth': 45.0, 'score': 0.7495301335789781}  
{'num_leaves': 32.0, 'min_data_in_leaf': 3.0, 'max_depth': 63.0, 'score': 0.7495301335789781}  
{'num_leaves': 32.0, 'min_data_in_leaf': 7.0, 'max_depth': 16.0, 'score': 0.752216860511667}  
{'num_leaves': 32.0, 'min_data_in_leaf': 7.0, 'max_depth': 22.0, 'score': 0.7508869753359529}  
{'num_leaves': 32.0, 'min_data_in_leaf': 7.0, 'max_depth': 31.0, 'score': 0.7499040284966618}  
{'num_leaves': 32.0, 'min_data_in_leaf': 7.0, 'max_depth': 45.0, 'score': 0.7499040284966618}  
{'num_leaves': 32.0, 'min_data_in_leaf': 7.0, 'max_depth': 63.0, 'score': 0.7499040284966618}  
{'num_leaves': 45.0, 'min_data_in_leaf': 2.0, 'max_depth': 16.0, 'score': 0.756357074789499}  
{'num_leaves': 45.0, 'min_data_in_leaf': 2.0, 'max_depth': 22.0, 'score': 0.7584757921973456}  
{'num_leaves': 45.0, 'min_data_in_leaf': 2.0, 'max_depth': 31.0, 'score': 0.759322815369111}  
{'num_leaves': 45.0, 'min_data_in_leaf': 2.0, 'max_depth': 45.0, 'score': 0.7550220449605046}  
{'num_leaves': 45.0, 'min_data_in_leaf': 2.0, 'max_depth': 63.0, 'score': 0.7550220449605046}  
{'num_leaves': 45.0, 'min_data_in_leaf': 3.0, 'max_depth': 16.0, 'score': 0.75624596877734}  
{'num_leaves': 45.0, 'min_data_in_leaf': 3.0, 'max_depth': 22.0, 'score': 0.759452539937531}  
{'num_leaves': 45.0, 'min_data_in_leaf': 3.0, 'max_depth': 31.0, 'score': 0.759581589916143}  
{'num_leaves': 45.0, 'min_data_in_leaf': 3.0, 'max_depth': 45.0, 'score': 0.753279594360448}  
{'num_leaves': 45.0, 'min_data_in_leaf': 3.0, 'max_depth': 63.0, 'score': 0.753279594360448}  
{'num_leaves': 45.0, 'min_data_in_leaf': 7.0, 'max_depth': 16.0, 'score': 0.7558170264053962}  
{'num_leaves': 45.0, 'min_data_in_leaf': 7.0, 'max_depth': 22.0, 'score': 0.7587351236045762}  
{'num_leaves': 45.0, 'min_data_in_leaf': 7.0, 'max_depth': 31.0, 'score': 0.7595214720070634}  
{'num_leaves': 45.0, 'min_data_in_leaf': 7.0, 'max_depth': 45.0, 'score': 0.7525010188021835}  
{'num_leaves': 45.0, 'min_data_in_leaf': 7.0, 'max_depth': 63.0, 'score': 0.7525010188021835}  
{'num_leaves': 63.0, 'min_data_in_leaf': 2.0, 'max_depth': 16.0, 'score': 0.7564967249892063}  
{'num_leaves': 63.0, 'min_data_in_leaf': 2.0, 'max_depth': 22.0, 'score': 0.7632724852260777}  
{'num_leaves': 63.0, 'min_data_in_leaf': 2.0, 'max_depth': 31.0, 'score': 0.7668477283752214}  
{'num_leaves': 63.0, 'min_data_in_leaf': 2.0, 'max_depth': 45.0, 'score': 0.766192487161526}  
{'num_leaves': 63.0, 'min_data_in_leaf': 2.0, 'max_depth': 63.0, 'score': 0.7604046241929917}  
{'num_leaves': 63.0, 'min_data_in_leaf': 3.0, 'max_depth': 16.0, 'score': 0.7590297726278221}  
{'num_leaves': 63.0, 'min_data_in_leaf': 3.0, 'max_depth': 22.0, 'score': 0.7643657905737826}  
{'num_leaves': 63.0, 'min_data_in_leaf': 3.0, 'max_depth': 31.0, 'score': 0.7663278921709152}  
{'num_leaves': 63.0, 'min_data_in_leaf': 3.0, 'max_depth': 45.0, 'score': 0.7647802560862802}  
{'num_leaves': 63.0, 'min_data_in_leaf': 3.0, 'max_depth': 63.0, 'score': 0.7597102497545686}  
{'num_leaves': 63.0, 'min_data_in_leaf': 7.0, 'max_depth': 16.0, 'score': 0.7579527214615925}  
{'num_leaves': 63.0, 'min_data_in_leaf': 7.0, 'max_depth': 22.0, 'score': 0.7621188884702516}  
{'num_leaves': 63.0, 'min_data_in_leaf': 7.0, 'max_depth': 31.0, 'score': 0.766407953406953}  
{'num_leaves': 63.0, 'min_data_in_leaf': 7.0, 'max_depth': 45.0, 'score': 0.762561166387289}  
{'num_leaves': 63.0, 'min_data_in_leaf': 7.0, 'max_depth': 63.0, 'score': 0.7594317113110272}  
{'num_leaves': 90.0, 'min_data_in_leaf': 2.0, 'max_depth': 16.0, 'score': 0.7593724028469931}  
{'num_leaves': 90.0, 'min_data_in_leaf': 2.0, 'max_depth': 22.0, 'score': 0.7654219544659256}  
  
2)  
n_estimators=100  
n_jobs=-1,  
random_state=r_state,  
class_weight='balanced',  
objective='binary',  
metric='binary_logloss',  
num_iterations=200  
learning_rate=0.10  
max_depth=32  
{'num_leaves': 32.0, 'min_data_in_leaf': 2.0, 'score': 0.7478428229679043}  
{'num_leaves': 32.0, 'min_data_in_leaf': 4.0, 'score': 0.7490562773369622}  
{'num_leaves': 45.0, 'min_data_in_leaf': 2.0, 'score': 0.7586257706315541}  
{'num_leaves': 45.0, 'min_data_in_leaf': 4.0, 'score': 0.7571181793489211}  
{'num_leaves': 63.0, 'min_data_in_leaf': 2.0, 'score': 0.7660735237681212}  
{'num_leaves': 63.0, 'min_data_in_leaf': 4.0, 'score': 0.7643192600524543}  
  
3)  
n_estimators=150  
n_jobs=-1,  
random_state=r_state,  
class_weight='balanced',  
objective='binary',  
metric='binary_logloss',  
num_iterations=500  
learning_rate=0.05  
max_depth=32  
num_leaves=64.0  
min_data_in_leaf=2.0  
{'score': 0.7722166540770777}  
  
4)  
n_estimators=150,  
n_jobs=-1,  
random_state=r_state,  
class_weight='balanced',  
objective='binary',  
metric='binary_logloss',  
num_iterations=500,  
learning_rate=0.05,  
boosting='gbdt',  
max_depth=32,  
max_bin=64,  
num_leaves=64,  
min_data_in_leaf=2  
{'score': 0.7746759038267277}  

### *CatBoost*

1)  
loss_function='Logloss',  
iterations=500,  
learning_rate=0.05,  
grow_policy='Depthwise',  
min_data_in_leaf=2,  
auto_class_weights='Balanced',  
thread_count=-1,  
logging_level='Silent'  
{'depth': array(\[ 4,  7, 16\]), 'border_count': array(\[ 61, 125, 254\])}  
{'depth': 4.0, 'border_count': 61.0, 'score': 0.6722395458543216}  
{'depth': 4.0, 'border_count': 125.0, 'score': 0.6708153885710613}  
{'depth': 4.0, 'border_count': 254.0, 'score': 0.6734939183595492}  
{'depth': 7.0, 'border_count': 61.0, 'score': 0.686461691048847}  
{'depth': 7.0, 'border_count': 125.0, 'score': 0.6839250574636615}  
{'depth': 7.0, 'border_count': 254.0, 'score': 0.6792443454278922}  
{'depth': 16.0, 'border_count': 61.0, 'score': 0.6873772407448846}  
{'depth': 16.0, 'border_count': 125.0, 'score': 0.6821022727272728}  
{'depth': 16.0, 'border_count': 254.0, 'score': 0.6892485346863791}  
  
2)  
loss_function='Logloss',  
iterations=200,  
learning_rate=0.10,  
grow_policy='Depthwise',  
min_data_in_leaf=2,  
auto_class_weights='Balanced',  
thread_count=-1,  
logging_level='Silent'  
{'depth': array(\[ 4,  6, 10, 16\]), 'border_count': array(\[ 61, 125, 254\])}  
{'depth': 4.0, 'border_count': 61.0, 'score': 0.6740169317192816}  
{'depth': 4.0, 'border_count': 125.0, 'score': 0.6841518027725557}  
{'depth': 4.0, 'border_count': 254.0, 'score': 0.6732768760291696}  
{'depth': 6.0, 'border_count': 61.0, 'score': 0.687623300182667}  
{'depth': 6.0, 'border_count': 125.0, 'score': 0.6787083350006002}  
{'depth': 6.0, 'border_count': 254.0, 'score': 0.6701201589198618}  
{'depth': 10.0, 'border_count': 61.0, 'score': 0.6836491298797285}  
{'depth': 10.0, 'border_count': 125.0, 'score': 0.6803100285771251}  
{'depth': 10.0, 'border_count': 254.0, 'score': 0.6824751580849142}  
{'depth': 16.0, 'border_count': 61.0, 'score': 0.7037387519141103}  
{'depth': 16.0, 'border_count': 125.0, 'score': 0.6932359589636585}  
{'depth': 16.0, 'border_count': 254.0, 'score': 0.692525114634744}  
  
3)  
loss_function='Logloss',  
iterations=200,  
learning_rate=0.10,  
grow_policy='Depthwise',  
min_data_in_leaf=2,  
auto_class_weights='Balanced',  
thread_count=-1,  
logging_level='Silent'  
{'depth': \[10, 16\], 'border_count': array(\[125, 254\])}  
{'depth': 10.0, 'border_count': 125.0, 'score': 0.7164429923635482}  
{'depth': 10.0, 'border_count': 254.0, 'score': 0.7152183454026773}  
{'depth': 16.0, 'border_count': 125.0, 'score': 0.7026980482204364}  
{'depth': 16.0, 'border_count': 254.0, 'score': 0.7103819176427157}  